# Ввод / Вывод, тип IO

Вспомним, что в Haskell нет понятия порядка вычислений. Результат не должен зависеть от того, в каком порядке вычисления происходили. Благодаря этому работают ленивые вычисления, значения вычисляются, когда они нужны. Теперь проблема, код на псевдоязыке

```
print "введите x"
x := read -- прочитать с клавиатуры
print "введите y"
y := read -- прочитать с клавиатуры
```

ожидает, что действия выполнятся друг за другом, будет странно, если сначала сработает ввод y.

В Haskell вводится тип `IO a`. Он означает значение `a`, которое будет получено только после операций ввода/вывода. Т.е. с `IO Int` нельзя работать как с числом, можно работать как с каким-то неизвестным числом, значение которого будет известно позже. Достать в коде программы целое значение из `IO Int` невозможно. В отличие от `Maybe a`, там достать значение можно, если оно есть.

Т.е. нет функции `f :: IO Int -> Int`.

Единственный способ достать значения, это только вернуть их из `main`, главной функции программы. `main` имеет тип `IO a`, и при запуске это значение внутри IO, наконец, вычислится и покажется на экране, например. Т.е. будут вычислены только те значения внутри IO, которые прямо или косвенно вызываются из `main`.

Примеры, давайте прочитаем с клавиатуры значения x, y и сложим их:

(отсупление, класс Read имеет функцию read, обратную show)

In [4]:
show 42
read "42" :: Int
read "asdf" :: Int -- невозможно прочитать

"42"

42

: 

In [11]:
import Control.Applicative

-- пытаемся прочитать число
x = readLn :: IO Int
-- пытаемся прочитать число
y = readLn :: IO Int

-- теперь давайте их сложим. Аналогично Maybe

z = liftA2 (+) x y -- или (+) <$> x <*> y

-- если вызвать z, будет два раза выполнен readLn, потом значения внутри сложатся

## Монады
Это расширение аппликативного функтора, структурный тип данных, который позволяет делать дополнительные действия: `>>=`.

In [12]:
:type (>>=)

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

Какой смысл для Maybe, т.е. для вычислений, в которых результата может не быть. Пусть есть функция `mbsqrt :: Double -> Maybe Double`, которая берет квадратный корень, но он не всегда может получиться.

In [15]:
mbsqrt :: Double -> Maybe Double
mbsqrt x = if x >= 0 then Just $ sqrt x else Nothing

Хотим вычислить с её помощью `f(x) = sqrt (sqrt(x) - 10) + 4`.
Как это записать с помощью `>>=`:

In [18]:
f :: Double -> Maybe Double
f x = (+4) <$> (mbsqrt x                   -- корень из x в коробочке
                   >>= (\sqx ->     -- sqx - это реальное числовое (Double) значение корня
                        mbsqrt (sqx - 10)
                        ))

In [22]:
f 100
f 4
f (-10)
f 10000

Just 4.0

Nothing

Nothing

Just 13.486832980505138

В принципе, можно пользоваться только `>==`:

In [28]:
f :: Double -> Maybe Double
f x = mbsqrt x                   -- корень из x в коробочке
            >>= (\sqx ->     -- sqx - это реальное числовое (Double) значение корня
                mbsqrt (sqx - 10)
                    >>= (\res -> return (res + 4))  -- return = Just для Maybe
                ) 
f 100
f 4
f (-10)
f 10000

Just 4.0

Nothing

Nothing

Just 13.486832980505138

Смысл `return` аналогичен `pure` для аппликативных функторов, он заворачивает значение в монаду.

## Do-нотация

Последний пример кода показывает, что мы можем с помощью `return` и `>>=` делать вычисления внутри Maybe, не разбирая случаи Just|Nothing. Но можно пойти дальше и ввести синтаксическую конструкцию, которая еще больше упростит написание:

In [27]:
f :: Double -> Maybe Double
f x = do
       sqx <- mbsqrt x
       res <- mbsqrt (sqx - 10)
       return (res + 4)
       
f 100
f 4
f (-10)
f 10000

Just 4.0

Nothing

Nothing

Just 13.486832980505138

do — это синтаксический сахар для выражений из `return` и `>>=`. Фактически, то, что мы написали для `f` через `do` заменяется на прошлую версию через `return` и `>>=`.

Посмотрим аналоичный пример для списков и для `IO`.
Для списков `>>=`. Давайте подумаем, как она может работать для списков.

`[10, 20, 30] >>= (\x -> [x-1, x+1])`

У нас есть значение, которое либо 10, либо 20, либо 30, мы из него либо вычитаем, либо добавляем 1.

In [29]:
[10, 20, 30] >>= (\x -> [x-1, x+1])

[9,11,19,21,29,31]

Фактически, для списков это `concatMap`.

Напишем функцию, которая добавляет или вычитает 1, потом умножает на 2 или на 3.

In [33]:
addOrSub1 :: Int -> [Int]
addOrSub1 x = [x - 1, x + 1]

mul2OrMul3 :: Int -> [Int]
mul2OrMul3 x = [2 * x, 3 * x]

f :: Int -> [Int]
f x = do
        y <- addOrSub1 x -- или y <- [x + 1, x - 1]
        z <- mul2OrMul3 y -- или [2 * y, 3 * y]
        return z
        
f 10

Line 8: Redundant return
Found:
do y <- addOrSub1 x
   z <- mul2OrMul3 y
   return z
Why not:
do y <- addOrSub1 x
   mul2OrMul3 y

[18,27,22,33]

In [35]:
f :: Int -> [Int]
f x = addOrSub1 x >>= (\y ->
         mul2OrMul3 y >>= ( \z ->
            return z
         )
      )
      
f 10

-- ИЛИ 

f :: Int -> [Int]
f x = addOrSub1 x >>= (\y ->
         mul2OrMul3 y >>= return -- упрощаем, \x -> f x эквивалентно f
      )
      
f 10

-- ИЛИ 

f :: Int -> [Int]
f x = addOrSub1 x >>= (\y ->
         mul2OrMul3 y -- упрощаем, x >= return эквивалентно x
      )
      
f 10

-- ИЛИ

f :: Int -> [Int]
f x = addOrSub1 x >>= mul2OrMul3 -- упрощаем, \x -> f x эквивалентно f
      
f 10



Line 2: Use >=>
Found:
\ y -> mul2OrMul3 y >>= (\ z -> return z)
Why not:
mul2OrMul3 Control.Monad.>=> (\ z -> return z)Line 3: Avoid lambda
Found:
(\ z -> return z)
Why not:
returnLine 13: Use >=>
Found:
\ y -> mul2OrMul3 y >>= return
Why not:
mul2OrMul3 Control.Monad.>=> returnLine 14: Monad law, right identity
Found:
mul2OrMul3 y >>= return
Why not:
mul2OrMul3 yLine 22: Avoid lambda
Found:
(\ y -> mul2OrMul3 y)
Why not:
mul2OrMul3

[18,27,22,33]

[18,27,22,33]

[18,27,22,33]

[18,27,22,33]

Предпоследнее в виде `do` выглядит так:

In [36]:
f :: Int -> [Int]
f x = do
        y <- addOrSub1 x -- addOrSub1 x -> (\y -> )
        mul2OrMul3 y
        
f 10

[18,27,22,33]

Подумать, как выглядит последнее через `do` или даже без `do`:

In [38]:
f :: Int -> [Int]
f x = addOrSub1 x >>= mul2OrMul3
        
f 10

[18,27,22,33]

С `IO` аналогично, но там бывают print, которые не возвращают интересных нам значений.